# Component contribution analysis

From the hyperparameter tuning step, we have selected a set of parameters ($\lambda$,  $ \vartheta$, k) with largest **LLPD$_o$**.  

Here we aim to evaluate the contribution of each of the components in the generative model (MEM). The generative model for the ocean microbiome data for the microbial abundance data consists of **geochemical component**, **spatio-temporal components indicating geographical location, ocean depth and time** and **species-species interaction component**. To understand their  contribution in the MEM, we drop each of the specific component in the generative model, estimate the variational posterior and then evaluate the model performance in terms of the **out of sample log-likelihood predictive density (LLPD$_o$)**.


Steps of the analysis
 + Define component excluded generative model
 + Python script for the variational posterior estimation: **component_contribution_fit.py**
 + Script to call component excluded generative model  
 + Evaluate the model in terms of the **LLPD$_o$**

 
#### Component excluded generative model
We have defined the component excluded stan model (see stan_model folder) in the following files:
 + **NB_microbe_ppc.stan** : Full model [Model = 0]
 + **NB_microbe_ppc-1.stan** : Province component dropped  [Model = 1]
 + **NB_microbe_ppc-2.stan** : Biome component dropped  [Model = 2]
 + **NB_microbe_ppc-3.stan** : Quarter/Time component dropped  [Model = 3]
 + **NB_microbe_ppc-G.stan** : Geochemical component dropped  [Model = 4]
 + **NB_microbe_ppc_nointer.stan** : Species-species interaction component dropped  [Model = 5]

#### Script to evaluate the model
For the chosen set of hyperparameters, we compute the variational posterior of each the **component excluded generative model** for **twenty** different initialization. We have saved the command calling **component_contribution_fit.py** for each of the  **component excluded generative model** in the file **mem_component_contribution** (see the python cript below to generate the file).

A line in the file **mem_component_contribution** calls the python script **component_contribution_fit.py** for a given choice of the parameters. 

*module purge ; module load slurm gcc python3 ; OMP_NUM_THREADS=1 python3 component_contribution_fit.py 100.0 0.219 0.06503 0.1 200 1 0 1.0 > logfile/1.log 2>&1*

Input parameters for a setting includes **latent rank (k),  $\lambda$,  $\vartheta$, test sample proportion, variational posterior sample size,  sub-settings unique ID (1-20), model type  and setting seed**. 


#### Parameter estimation 
We run the script on server using the command:
*sbatch -N [#node] -p [#partition] disBatch.py -t [#task on each node] [script_file]*
Example: *sbatch -N 10 -p ccm disBatch.py -t 25 mem_component_contribution*




#### Model output analysis
Evaluate each of the **component excluded generative model** based on the $LLPD_o$. Let us consider our model output is saved in the folder **CContribution**. To compare the model, we load the output file and compute the $LLPD_o$ for each model. 

In [1]:
# load python module 
import os
import numpy as np
import random
import pickle as pkl
import glob
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# output file name 
# We have save the output in [CContribution] folder
fname_o = glob.glob('CContribution/*model_nb_cvtest.pkl')
out = np.empty((len(fname_o),7))
mtype_file = np.argsort([int(x.split('/')[1].split('_')[1].split('.')[0]) for x in fname_o])
fname_o = [fname_o[i] for i in mtype_file]
#fname_o[2]

In [3]:
# Extract model output and compute LLPD
for i in range(len(fname_o)):
    if ((i%20) == 0):
        print(i)
    [holdout_mask, Yte_sample, llpd, n_test, l,m_seed,sp_mean,\
                 sp_var, h_prop, uid, nsample_o, Yte_fit,\
                 cv_test,Y, muest, Yte_cv, Yte_lpmf, kl_comp] = pkl.load(open(fname_o[i], "rb"))
    se_index  = holdout_mask == 1. 

    # LLPD compute 
    temp_ll = cv_test[se_index]
    temp_ll = np.mean(temp_ll)  
    mtype = fname_o[i].split('/')[1].split('_')[1]
    mtype = int(mtype.split('.')[0])
    error = np.mean(np.power(Y - muest,2)[se_index])
    out[i] = [i, l, sp_mean,sp_var,  temp_ll,uid, mtype]


0
20
40
60
80
100


In [7]:
pkl.dump(out, open('comparison_model', "wb"))  # save output 
out = pkl.load(open('comparison_model', "rb"))
out = pd.DataFrame(data=out)
out.columns = ['Index','rank','lambda','upsilon','llpd','uid','Model']
out.head(20)

,Index,rank,lambda,upsilon,llpd,uid,Model
0,0.0,200.0,0.246,0.10063,-3.347054,5.0,0.0
1,1.0,200.0,0.246,0.10063,-3.341781,8.0,0.0
2,2.0,200.0,0.246,0.10063,-3.320363,14.0,0.0
3,3.0,200.0,0.246,0.10063,-3.324665,3.0,0.0
4,4.0,200.0,0.246,0.10063,-3.333003,11.0,0.0
5,5.0,200.0,0.246,0.10063,-3.339617,4.0,0.0
6,6.0,200.0,0.246,0.10063,-3.337089,7.0,0.0
7,7.0,200.0,0.246,0.10063,-3.347768,2.0,0.0
8,8.0,200.0,0.246,0.10063,-3.316317,1.0,0.0
9,9.0,200.0,0.246,0.10063,-3.349977,16.0,0.0


<font color=blue>**Our analysis suggest that the model 0, i.e., MEM is best with highest out of sample LLPD$_o$.** </font>

In [5]:
out.groupby(['Model'], as_index=False).agg({'llpd':['mean','std']})

Model      llpd          
             mean       std
0   0.0 -3.332235  0.013148
1   1.0 -3.337667  0.013125
2   2.0 -3.355374  0.015495
3   3.0 -3.339841  0.009076
4   4.0 -3.388219  0.010832
5   5.0 -3.333542  0.010483

In [13]:
out_sub = out.groupby(['Model'], as_index=False).mean()[['Model','llpd']].transpose()
print(out_sub.to_latex(float_format="%.4f", bold_rows = True))

\begin{tabular}{lrrrrrr}
\toprule
{} &       0 &       1 &       2 &       3 &       4 &       5 \\
\midrule
\textbf{Model} &  0.0000 &  1.0000 &  2.0000 &  3.0000 &  4.0000 &  5.0000 \\
\textbf{llpd } & -3.3322 & -3.3377 & -3.3554 & -3.3398 & -3.3882 & -3.3335 \\
\bottomrule
\end{tabular}



In [14]:
l,m_seed,sp_mean, sp_var

(200, 3, 0.246, 0.10063)